# Information Retrieval Project
- 1. Get all the training model datasets from online
- 2. Change them to postive, negative, neutral
- 3. Combine all the datasets together
-----------------------------
- 4. Remove punctuation,stopwords, do lemmatization & stemming
- 5. Get tf-idf
- 6. Choose a machine learning model/deep learning model
    - ML: SVM,		Random Forest,		Logistic Regression(this),		Gradient Boosting 	Naive Bayes

    - DL: use with word embeddings like Word2Cev or GloVe to get semantic relationships between words
RNN		LSTM		CNN	   BERT(this)

- 7. Concatenate BERT value with TFIDF value
----------------------------------
- 8. Train Test Split using TFIDF values as Features and sentiment labels as Targets
- 9. Evaluate Accuracy, Precision, Recall, F1-score of Model using Logistic Regression
----------------------------------
- 10. Test Model on Manually Labelled data after Scraping from Web
- 11. Preprocess manually labeled data in the same way as online datasets
- 12. Compare and Calculate Accuracy of Model
---------------------------------
- 13. Identify strengths and weaknesses of model
- 14. Make appropriate adjustments
- 15. Report and recommendations

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ZW\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZW\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ZW\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# TF-IDF


In [ ]:
def preprocess_text(text):

  # Remove special characters
  text = re.sub(r'[^a-zA-Z0-9\s]','', text) # pattern matches any character not alphanumeric

  # Tokenisation
  tokens = word_tokenize(text)

  # Remove punctuation
  tokens = [token for token in tokens if re.match(r'\b\w+\b',token)]

  # Remove stopwords
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token.lower() not in stop_words]

  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(token) for token in tokens]

  # Stemming
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(token) for token in tokens]

  return ' '.join(tokens)


# Read the CSV file
df = pd.read_csv('train_pos_neg.csv')

# Preprocess the 'Text' column
df['preprocessed_text'] = df['Text'].apply(preprocess_text)

# Calculate TF-IDF for each preprocessed headline
tfidf = TfidfVectorizer(sublinear_tf=True,ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(df['preprocessed_text'])

# Get feature names (words)
feature_names = tfidf.get_feature_names_out()

tfidf_matrix_dense = tfidf_matrix.toarray()

tfidf_df = pd.DataFrame(tfidf_matrix_dense,columns = feature_names)

# Print TF-IDF scores for each word in each text
for i,row in enumerate(tfidf_matrix_dense):
  non_zero_indices = row.nonzero()[0]
  print(f"Non-zero TF-IDF values for row{i+ 1}:")
  for index in non_zero_indices:
    word = feature_names[index]
    score = row[index]
    print(f"{word}: {score:.6f}")
  print()

Streaming output truncated to the last 5000 lines.
manufactur scanfil: 0.209125
mn: 0.197984
mn 2005: 0.201137
mn 2006: 0.204797
net: 0.112089
net sale: 0.128385
report: 0.114520
report net: 0.183045
sale: 0.100625
sale eur: 0.180074
scanfil: 0.183045
scanfil report: 0.221251

Non-zero TF-IDF values for row12141:
2007: 0.125375
2007 eur: 0.201387
589: 0.206488
589 mn: 0.222333
624: 0.222333
624 mn: 0.222333
contract: 0.128546
contract manufactur: 0.176272
earlier: 0.137440
electron: 0.155632
electron contract: 0.201387
eur: 0.168478
eur 589: 0.213064
eur 624: 0.222333
finnish: 0.108328
finnish electron: 0.181373
manufactur: 0.137720
manufactur scanfil: 0.201387
mn: 0.190658
mn second: 0.181373
mn year: 0.166500
net: 0.107942
net sale: 0.123634
quarter: 0.113962
quarter 2007: 0.162004
report: 0.110282
report net: 0.176272
sale: 0.096902
sale eur: 0.173411
scanfil: 0.176272
scanfil report: 0.213064
second: 0.133600
second quarter: 0.151671
year: 0.096949
year earlier: 0.146989

Non-zero 

# BERT word embeddings
-  Install Transformers via pip: pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel,pipeline
import torch
import pandas as pd

# Load pre trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Define a function for BERT preprocessing using pipeline
bert_pipeline = pipeline('feature-extraction',model=model,tokenizer=tokenizer)

print("here first")
def bert_preprocess_text(text):
  # Preprocess the text using BERT pipeline
  embeddings = bert_pipeline(text)

  # Use mean pooling to get a single embedding for the text
  return torch.mean(torch.tensor(embeddings[0]), dim=0).numpy()

counter = 0

# Convert text to word embeddings to create a new DataFrame
embeddings_list = []
for text in df['Text']:
  embeddings = bert_preprocess_text(text)
  embeddings_list.append(embeddings)
  print(counter)
  counter+=1
df_embeddings = pd.DataFrame(embeddings_list)

Streaming output truncated to the last 5000 lines.
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398
7399
7400
7401
7402
7403
7404
7405
7406
7407
7408
7409
7410
7411
7412
7413
7414
7415
7416
7417
7418
7419
7420
7421
7422
7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507

# Combined TF-IDF and BERT word embeddings

In [ ]:
# Combine together tfidf and word embeddings into one DataFrame
df_combined = pd.concat([tfidf_df,df_embeddings],axis=1)

# Train model with Train Test Split and self test model
- 0: negative
- 1: positive

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [ ]:
# Separate features(TF-IDF and word embeddings) and target variable
X = df_combined
y = df['Sentiment']
print("step 1 done")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                    random_state=42)
print("step 2 done")

#Convert feature names to strings
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

##Initialise and train a logistic regression model for multiclass classification
logreg_model = LogisticRegression(max_iter=1000)
  # or 'ovr' for one-vs-rest
logreg_model.fit(X_train, y_train)
print("step 3 done")

print("step 4 done")

print("step 5 done")

## Make predictions on the testing data
y_pred = logreg_model.predict(X_test)

print("step 6 done")

## Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test,y_pred)
print("step 7 done")

print(f'Accuracy: {accuracy}')
print(f'Classification report:\n{classification_report}')

step 1 done
step 2 done
step 3 done
step 4 done
step 5 done
step 6 done
step 7 done
Accuracy: 0.8153409090909091
Classification report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75       911
           1       0.85      0.86      0.85      1553

    accuracy                           0.82      2464
   macro avg       0.80      0.80      0.80      2464
weighted avg       0.82      0.82      0.82      2464



# Test on manually labelled data

# TF-IDF

In [ ]:
# Read the manually labelled data
df_manual = pd.read_csv('test_pos_neg.csv')

# Preprocess the 'Headline column in df_manual
df_manual['preprocessed_text'] = df_manual['Headline'].apply(preprocess_text)

# Calculate TF-IDF for each preprocessed headline in df_manual
tfidf_manual = tfidf.transform(df_manual['preprocessed_text'])

# Get feature names(words)
feature_names_manual = tfidf.get_feature_names_out()

# Convert TF-IDF values to a DataFrame
tfidf_matrix_dense_manual = tfidf_manual.toarray()
tfidf_df_manual = pd.DataFrame(tfidf_matrix_dense_manual, columns=feature_names_manual)

# BERT word embeddings

In [ ]:
# Load pre trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Define a function for BERT preprocessing using pipeline
bert_pipeline = pipeline('feature-extraction',model=model,tokenizer=tokenizer)

print("here first")

counter = 0

# Convert text to word embeddings to create a new DataFrame
embeddings_list = []
for text in df_manual['Headline']:
  embeddings = bert_preprocess_text(text)
  embeddings_list.append(embeddings)
  print(counter)
  counter+=1

df_embeddings_manual = pd.DataFrame(embeddings_list)

here first
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

# Combined TF-IDF with BERT word embeddings

In [ ]:
# Concatenate TF-IDF values with word embeddings
df_combined_manual = pd.concat([tfidf_df_manual, df_embeddings_manual], axis=1)

# Perform Classification using previously trained and saved model
- 0: negative
- 1: positive

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [ ]:
# Read the CSV file
df_manual = pd.read_csv('test_pos_neg.csv')
print("1 file imported")

1 file imported


In [ ]:
#Convert feature names to strings
df_combined_manual.columns = df_combined_manual.columns.astype(str)

## Make predictions using the trained model
##-for logistic regression only
y_pred_manual = logreg_model.predict(df_combined_manual)
print("step 1 done")

y_test_manual = df_manual['FINAL Sentiment']

## Evaluate the model
##-for just logistic regression
accuracy = accuracy_score(y_test_manual, y_pred_manual)
report = classification_report(y_test_manual,y_pred_manual)
print("step 2 done")

print(f'Accuracy: {accuracy}')
print(f'Classification report:\n{report}')

step 1 done
step 2 done
Accuracy: 0.7741935483870968
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.67      0.75       601
           1       0.73      0.87      0.80       608

    accuracy                           0.77      1209
   macro avg       0.79      0.77      0.77      1209
weighted avg       0.79      0.77      0.77      1209

